In [3]:
import json

In [394]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

oai_client = OpenAI(api_key =os.getenv("LANDPRO_OAI_KEY"))

load_dotenv('/Users/paigegiese/SYG/landproDATA_code/misc-work/.env')

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/landprodata")
query = "SELECT * FROM instruments"
df = pd.read_sql(query, con=engine)
df.head()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x104a84a10>>
Traceback (most recent call last):
  File "/Users/paigegiese/SYG/landproDATA_code/harvest-1/landproDATA/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


MySQLInterfaceError: Lost connection to MySQL server during query

In [4]:
with open("entities_reclassed_v1.jsonl", "r") as f:
    data = [json.loads(line) for line in f if line.strip()]

In [9]:
reclassified = pd.DataFrame()
for d in data:
    reclassified = pd.concat([reclassified, pd.DataFrame(d)]).reset_index(drop=True)

In [13]:
df[['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']]

,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name
0,1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying
1,2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.
2,3,Firm,Developer,RE - Development,A Team Land Consultants LLC,A TEAM LAND CONSULTANTS LLC
3,4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C."
4,5,Agency,Public Agency,,Ada County Highway District,
...,...,...,...,...,...,...
13390,16309,None,,,Self,None
13391,16311,None,,,landproDATA Inc.,None
13392,16312,None,,,AMP,None
13393,16313,None,,,"landproDATA, Inc",None


In [22]:
reclassified2 = reclassified.loc[~reclassified['original_id'].isin(reclassified['original_id'].value_counts().head(3).index.tolist())].reset_index(drop=True)
reclassified2['original_id'] = reclassified2['original_id'].astype(int)

In [25]:
reclassified3 = reclassified2.merge(df[['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']], how='inner', left_on='original_id', right_on='ID')

In [27]:
reclassified3['entity_group_id'].value_counts()

entity_group_id
Private    7804
Public      184
Name: count, dtype: int64

In [28]:
reclassified3['entity_category'].value_counts()

entity_category
Contractor                4514
Real Estate                752
Builder                    693
Other                      616
Engineering                406
Surveying                  291
Agency                     196
Architect                  134
Appraisal                  134
Inspection                  50
Supply                      39
Design                      28
Homeowners Association      24
Remodeling                  18
Restoration                 17
Title                       16
Landscape                   15
Environmental               12
Electrical                  11
Sign                         6
Communications               6
Trucking                     3
Developer                    2
Stone                        1
Testing                      1
Excavation                   1
Security                     1
Planning                     1
Name: count, dtype: int64

In [30]:
reclassified3['entity_subgroup_id'].value_counts()[:20]

entity_subgroup_id
General         2291
NULL            1507
Residential      907
Electrical       429
Plumbing         277
                 244
HVAC             227
Landscape        171
Roofing          133
Excavation       122
Water            102
Civil            101
Commercial        81
Remodeling        71
Mechanical        63
Full Service      60
Irrigation        57
Concrete          56
Fire              48
Home              47
Name: count, dtype: int64

In [51]:
reclassified3 = reclassified3.replace('',None).replace('NULL',None)

In [58]:
reclassified3['entity_category'].value_counts()

entity_category
Contractor                4514
Real Estate                752
Builder                    693
Other                      616
Engineering                406
Surveying                  291
Agency                     196
Architect                  134
Appraisal                  134
Inspection                  50
Supply                      39
Design                      28
Homeowners Association      24
Remodeling                  18
Restoration                 17
Title                       16
Landscape                   15
Environmental               12
Electrical                  11
Sign                         6
Communications               6
Trucking                     3
Developer                    2
Stone                        1
Testing                      1
Excavation                   1
Security                     1
Planning                     1
Name: count, dtype: int64

In [59]:
rules = [{'Entity Category': 'Surveying', 'Entity Subgroup ID': "Subgroup shall be left NULL",'commentary': "will be separate from engineering unless included within an Engineering. See Engineering Category rule."},
{'Entity Category': 'Contractor', 'Entity Subgroup ID': '<insert specialty here>. If no specialty, denote "General"','commentary': "Shall include records that indicate construction as a general contractor or sub contractor. \
 In the case of a specialty contracting, like 'HVAC', specify the specialty in the 'Entity Subgroup ID' key.\
 If 'Construction' is in the business name, Entity Category is 'Contractor', regardless of 'Builder' being present in either the Entity Subgroup ID or Entity Category of the INPUT DATA."},
{'Entity Category': 'Builder', 'Entity Subgroup ID': "'Residential', 'Commercial', or NULL if both. If 'Homes' is in the compny name, it is safe to assume 'Residential' as long as there is 0 indication of commercial activity.",'commentary': "Home builders are denoted with 'Residential' subgroup. If there is no indication of being a Builder in the input data, defer the classification to Contractor."},
{'Entity Category': 'Engineering', 'Entity Subgroup ID': "Insert specialty(ies) if known. 'Full service', if known.",'commentary': "Shall include all different engineering professions and specialities. The subgroup will be 'Full Service' if specified in input data. Denote the specialty (e.g. 'Civil') in Subgroup ID. If multiple, separate the specialities using commas."},
{'Entity Category': 'Real Estate', 'Entity Subgroup ID': "Subgroup will be NULL unless otherwise specified as 'Residential','Commercial','Land', or a comma separated combination of those values. 'Land' can be used as a Subgroup to identify Land Brokers.",'commentary': "Expand 'RE' in the input data to 'Real Estate'"},
{'Entity Category': 'Agency', 'Entity Subgroup ID': "Attempt the following valid values: 'Water', 'Road', Assessor', 'Municipality', 'Land', 'Health', 'Urban Renewal', 'Fire', 'Ancillary'. ONLY use Ancillary if no other category makes sense in context.",'commentary': "NOTE: all Agencies should be marked 'Public' in the Entity Group ID key."},
{'Entity Category': 'Title', 'Entity Subgroup ID': "NULL",'commentary': "Used to identify Private title companies."},
{'Entity Category': 'Architect', 'Entity Subgroup ID': "NULL unless 'Landscape', then place 'Landscape' as the Subgroup.",'commentary': ""},
{'Entity Category': 'Inspection', 'Entity Subgroup ID': "Attempt the following value values: 'Home','Electrical','Fire','Health','Other'. Use 'Other' if none of the valid values make sense.",'commentary': ""},
{'Entity Category': 'Appraisal','Entity Subgroup ID': "NULL", 'commentary':"you will find 'Appraiser' often in the input data"},
{'Entity Category': 'Homeowners Association','Entity Subgroup ID': "NULL", 'commentary':"Note that values 'HOA' are equivalent. Translate to 'Homeowners Association'"},
{'Entity Category': 'Other','Entity Subgroup ID': "NULL", 'commentary':"Use only in the event that you are not confident in the reclassification."}]
cats =[]
for r in rules:
    cats.append(r['Entity Category'])

In [62]:
cats= cats[:-1]

In [63]:
for i, j in reclassified3.iterrows():
    if j['entity_category'] not in cats:
        reclassified3.at[i,'entity_category'] = 'Other'

In [71]:
reclassified3['entity_subgroup_id'].value_counts(normalize=True).cumsum()[:20]

entity_subgroup_id
General         0.367324
Residential     0.512747
Electrical      0.581530
Plumbing        0.625942
HVAC            0.662338
Landscape       0.689755
Roofing         0.711079
Excavation      0.730640
Water           0.746994
Civil           0.763187
Commercial      0.776174
Remodeling      0.787558
Mechanical      0.797659
Full Service    0.807279
Irrigation      0.816418
Concrete        0.825397
Fire            0.833093
Home            0.840629
Municipality    0.848004
Land            0.854738
Name: proportion, dtype: float64

Add backs from original

In [65]:
not_in = df.loc[~df['ID'].isin(reclassified3['ID'].tolist())][['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']].replace('', None)

for c in ['original_id', 'entity_group_id', 'entity_category',
       'entity_subgroup_id', 'confidence', 'candidates']:
       not_in[c] = None

full_table = pd.concat([reclassified3, not_in[['original_id', 'entity_group_id', 'entity_category',
       'entity_subgroup_id', 'confidence', 'candidates', 'ID',
       'Entity Group ID', 'Entity Subgroup ID', 'Entity Category', 'Name',
       'IDSOS Name']]])

/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_28262/1817622094.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_table = pd.concat([reclassified3, not_in[['original_id', 'entity_group_id', 'entity_category',


### Status Determination

In [79]:
# 0 = declined to process for quality issues (i.e. should be removed) .. includes Landpro records and inactives
# 1 = declined to process for 'developer' review = audit
# 2 = declined to process because of too little initial information (null,null) = audit
# 3 = declined to process because they were marked 'consultant' = audit 
# 4 = failed to process in first run -> needs re-run
# 5 = builder -> not builder = audit
# 6 = entity_category marked 'Other' = audit
# 7 = processed and leaving in... = no audit

full_table['status'] = np.nan

# 1
devs=(full_table['Entity Category'].fillna('').str.contains('eveloper')
                    | full_table['Entity Subgroup ID'].fillna('').str.contains('eveloper'))
# 2
nullnull = (full_table['Entity Category'].isna() & full_table['Entity Subgroup ID'].isna())

# 0
inactive = (full_table['Name'].fillna('').str.contains('INACTIVE') |full_table['Name'].fillna('').str.contains('INATIVE'))

# 3
consultants=(full_table['Entity Category'].fillna('').str.contains('onsultant')
                    | full_table['Entity Subgroup ID'].fillna('').str.contains('onsultant')
                    | full_table['Name'].fillna('').str.contains('onsultant'))
# 0
lpd = full_table['Name'].fillna('').str.lower().str.contains('landpro')

# 5
not_builders = (((full_table['Entity Category']=='Builder') | (full_table['Entity Subgroup ID']=='Builder')) & ((full_table['entity_category']!= 'Builder') & full_table['entity_category'].notnull()))

full_table.loc[devs, 'status'] = 1


In [80]:
full_table.loc[nullnull, 'status'] = 2

In [81]:
full_table.loc[inactive, 'status'] = 0

In [82]:
full_table.loc[consultants, 'status'] = 3

In [83]:
full_table.loc[lpd, 'status'] = 0
full_table.loc[not_builders, 'status'] = 5

In [84]:
full_table['status'].value_counts()

status
2.0    2161
0.0    1899
1.0    1136
5.0     376
3.0      40
Name: count, dtype: int64

In [ ]:
full_table.loc[(full_table['status'].isna() & full_table['entity_category'].isna()), 'status'] = 4

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
0,1,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying,NaN
1,2,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.,NaN
2,4,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C.",NaN
3,5,Public,Agency,Road,99.0,"[{'entity_group_id': 'Public', 'entity_categor...",5,Agency,Public Agency,None,Ada County Highway District,None,NaN
4,6,Public,Agency,Land,95.0,"[{'entity_group_id': 'Public', 'entity_categor...",6,Agency,Public Agency,None,Ada County Surveyor's Office,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7983,15757,Private,Engineering,Full Service,98.0,[],15757,Firm,Engineering - Full Service,Engineering - Full Service,Project Delivery Group (PDG),None,NaN
7984,15758,Private,Engineering,None,95.0,[],15758,Firm,Engineering,Engineering,Raymond J Briggs and Associates,None,NaN
7985,15794,Private,Engineering,"Civil, Planning, Surveying",90.0,"[{'entity_group_id': 'Private', 'entity_catego...",15794,Firm,"Engineer, Planner, Surveyor",None,Wilding Engineering,Wilding Engineering Inc.,NaN
7986,15801,Private,Surveying,None,95.0,"[{'entity_group_id': 'Private', 'entity_catego...",15801,Firm,"Surveying, Mapping",Survey,Enclosure Surveying,None,NaN


In [97]:
full_table.loc[(full_table['entity_category']=='Other')
| (full_table['confidence']<=70) & (full_table['entity_category'].fillna('')!= 'Other'),'status'] = 6

In [104]:
full_table.loc[full_table['status'].isna(),'status'] = 7

In [127]:
print(full_table['status'].value_counts())
print(full_table['status'].isin([5]).sum())

status
7.0    6788
2.0    2161
0.0    1899
1.0    1136
6.0     842
5.0     358
4.0     171
3.0      40
Name: count, dtype: int64
358


In [144]:
print(full_table.loc[(full_table['status']==1) & (full_table['Entity Category'].str.contains('eveloper'))]['Entity Category'].value_counts())
print(' ')
print(full_table.loc[(full_table['status']==1) & (full_table['Entity Subgroup ID'].str.contains('eveloper'))]['Entity Subgroup ID'].value_counts())

full_table.loc[(full_table['status']==1) & (full_table['Entity Category'].str.contains('eveloper'))]['Entity Category'].value_counts().index.tolist() +\
    full_table.loc[(full_table['status']==1) & (full_table['Entity Subgroup ID'].str.contains('eveloper'))]['Entity Subgroup ID'].value_counts().index.tolist()

Entity Category
Developer                    277
Builder/Developer             31
RE - Commercial/Developer      6
Builder, Developer             6
Builder Developer              2
Developer,Builder              2
Name: count, dtype: int64
 
Entity Subgroup ID
Developer                     945
Builder, Developer             72
Developer, Builder             42
Developer/Investments           8
Owner, Developer                6
Developer, Engineer             5
Developer, Owner                2
Builder, Developer, CommRE      1
Name: count, dtype: int64


['Developer',
 'Builder/Developer',
 'RE - Commercial/Developer',
 'Builder, Developer',
 'Builder Developer',
 'Developer,Builder',
 'Developer',
 'Builder, Developer',
 'Developer, Builder',
 'Developer/Investments',
 'Owner, Developer',
 'Developer, Engineer',
 'Developer, Owner',
 'Builder, Developer, CommRE']

## Filling in some data where we can easily see the categories...

Clearly contractors, and not home builders

In [ ]:
full_table.loc[full_table['status'].isin([2,4]) & (full_table['Name'].str.contains('onstruction')) & (~full_table['Name'].fillna('').str.lower().str.contains('homes'))].reset_index(drop=True)

In [191]:
full_table[['entity_group_id','entity_category','entity_subgroup_id']].value_counts()

entity_group_id  entity_category  entity_subgroup_id                        
Private          Contractor       General                                       2254
                 Real Estate      Residential                                    512
                 Contractor       Electrical                                     428
                 Builder          Residential                                    393
                 Contractor       Plumbing                                       277
                                                                                ... 
                                  Power                                            1
                                  Plumbing, Septic                                 1
                                  Plumbing, Electrical                             1
                                  Plumbing, Backflow                               1
Public           Engineering      Full Service, Surveying, Mapping, Plann

In [ ]:
full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes')),'original_id'] = full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes'))]['ID']

full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes')),'entity_group_id'] = 'Private'

full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes')),'entity_category'] = 'Contractor'

full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes')),'entity_subgroup_id'] = 'General'

full_table.loc[full_table['status'].isin([2,4]) 
& (full_table['Name'].str.contains('onstruction')) 
& (~full_table['Name'].fillna('').str.lower().str.contains('homes')),'status'] = 7


Dirty data.. 

In [209]:
full_table.loc[full_table['Name'].fillna('').str.contains('@'),'status'] = 0 

Homes builders not classified (note: I deleted the contractor manual cell - but that filter is intended to remove contractors, leave home builders, and denote them as home builders)

In [ ]:
full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes')),'original_id'] = full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes'))]['ID']

full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes')),'entity_group_id'] = 'Private'

full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes')),'entity_category'] = 'Builder'

full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes')),'entity_subgroup_id'] = 'Residential' 


,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
2187,2664,Private,Builder,Residential,NaN,None,2664,Firm,Builder,None,"Broadway Homes, Inc.","BROADWAY HOMES, INC.",4.0
2230,2707,Private,Builder,Residential,NaN,None,2707,Firm,Builder,None,Shadowcreek Homes,SHADOWCREEK HOMES,4.0
2790,3298,Private,Builder,Residential,NaN,None,3298,Firm,Contractor,None,American Dream Homes and Design,AMERICAN DREAM HOMES AND DESIGN,4.0
10775,13065,Private,Builder,Residential,NaN,None,13065,Firm,None,None,JS Custom Homes LLC,JS Custom Homes LLC,2.0
10784,13074,Private,Builder,Residential,NaN,None,13074,Firm,None,None,ABY Homes LLC,ABY Homes LLC,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13364,16277,Private,Builder,Residential,NaN,None,16277,Firm,None,None,Eternity Homes LLC,Eternity Homes LLC,2.0
13365,16278,Private,Builder,Residential,NaN,None,16278,Firm,None,None,Evanow Homes and Construction LLC,Evanow Homes and Construction LLC,2.0
13378,16294,Private,Builder,Residential,NaN,None,16294,Firm,None,None,Homes LLC,None,2.0
13380,16296,Private,Builder,Residential,NaN,None,16296,Firm,None,None,JV Homes LLC,None,2.0


In [214]:
full_table.loc[(full_table['status'].isin([4,2]))
& (~full_table['ID'].isin(contractor_manual['original_id'].tolist()))
& (full_table['Name'].str.lower().str.contains('homes')),'status'] = 7

In [230]:
full_table.loc[full_table['status'].isin([2])]['Name'].sample(10)

12808                       Upland Engineers
11198                  Steady Earthworks LLC
11658    Unlimited Solutions Landscaping LLC
13240         Silvercreek Real Estate Group 
12466                        DC DirtWorx LLC
12525                 Steadfast Builders LLC
12956                Broadbent Ventures, LLC
12569            Alliance Serving Kuna, Inc.
12657                      RB-AMS Eagle, LLC
10839                                    XYZ
Name: Name, dtype: object

## Need reprocessed because they errored in initial run OR had null, null in initial run, and now we're trying just based on name... 

In [231]:
ready_to_reclass_r2 = full_table.loc[(full_table['status'].isin([4,2]))][['ID','Name','Entity Group ID','Entity Category','Entity Subgroup ID']].rename(columns={'ID':'original_id'}).to_dict('records')

In [240]:
len(ready_to_reclass_r2)

1886

In [241]:
categories = set(full_table['entity_category'])
subgroups = set(full_table['entity_subgroup_id'])

In [235]:
full_table.loc[full_table['status'].isin([4,2])][['ID','Name','Entity Group ID','Entity Category','Entity Subgroup ID']].rename(columns={'ID':'original_id'}).to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/db/entities_to_be_reclassed_06162025.csv', index=False)

## Process in mini batches (sets of 5)

In [238]:
from typing import TypedDict, List, Dict, Any, Optional
from pydantic import BaseModel, Field
from typing import TypedDict, List, Dict, Any, Optional, Annotated, Literal
oai_client = OpenAI(api_key =os.getenv("LANDPRO_OAI_KEY"))

#### Prompts

In [236]:
rules = json.dumps(rules)

In [239]:
class Candidate(BaseModel):
    entity_group_id: Annotated[Literal['Public','Private'], Field(description="Input key: 'Entity Group ID'. Apply rules in the system prompt.")]
    entity_category:Annotated[str, Field(description="Input key: 'Entity Category'. Apply rules in the system prompt.")]
    entity_subgroup_id:Annotated[str, Field(description="Input key: 'Entity Subgroup ID'. Apply rules in the system prompt. Be as consistent as possible.")]
    confidence: Annotated[float, Field(description="Based on the input data, how confident are you that this should be a candidate classification? 0-100 scale only.")]
    reasoning: Annotated[str, Field(description="Reasoning for evaluating a secondary classification to be a valid candidate.")]

class Reclassify(BaseModel):
    original_id: Annotated[str, Field(description="An exact copy of original_id from the input.")]
    entity_group_id: Annotated[Literal['Public','Private'], Field(description="Input key: 'Entity Group ID'. Apply rules in the system prompt.")]
    entity_category:Annotated[str, Field(description="Input key: 'Entity Category'. Apply rules in the system prompt.")]
    entity_subgroup_id:Annotated[str, Field(description="Input key: 'Entity Subgroup ID'. Apply rules in the system prompt. Be as consistent as possible.")]
    confidence: Annotated[float, Field(description="Based on the input data, how confident are you that this should be the classification? 0-100 scale only.")]
    candidates: Annotated[List[Candidate], Field("Returns a list of candidate classifications, based on multiple rules evaluating to True.")] 

class ReclassifyList(BaseModel):
    reclassified: Annotated[List[Reclassify], Field("Preserve the list in the SAME ORDER as the input list")]

system_prompt = f""" ## Role\
You are data cleaning assistant tasked with cleaning our entities information. It's incredibly important that you adhere to the rules as outlined below, applying them precisely and \
    giving indication as to your degree of uncertainty when applicable.

## Understand the input
You will receive input in the following format. IGNORE THE KEY 'original_id' in interpretation.Copy it directly to the output object for tracking purposes.

  'original_id': 1,
  'Entity Group ID': 'Firm',
  'Entity Subgroup ID': 'Surveyor',
  'Entity Category': 'Survey',
  'Name': '2 Bar K Land Surveying'


## Understanding the meaning of common formatting in the input data
- RE refers to 'Real Estate'. All instances of "RE" should be expanded to 'Real Estate' for reclassification
- Categories and subgroups may be combined in the input data. For example 'Engineer - Civil' would reclassify to: Entity Group ID: 'Private', Entity Category: 'Engineering', Entity Subgroup ID: 'Civil'
- You will come across spelling and formatting errors in the input. Remove apostrophes in values. 
    
## Rules\
Use the context and semantic meaning from the 4 input keys to correctly re-classify Entity Group ID, Entity Subgroup ID, and Entity Category.
! IMPORTANT ! : If you cannot clearly/easily tell from the Name what the classification should be, and you have no other data, return the following:
  
  'original_id': <'original_id'>,
  'Entity Group ID': 'Private',
  'Entity Subgroup ID': NULL,
  'Entity Category': 'Other',
  'Name': <'Name'>

Weight your responses to be bias to the existing data. Do not inject overly creative speculation into your evaluation. 
In general, determine the correct Entity Group ID, then Entity Category, then define the Subgroup. Follow the values provided below with precision.

Here are the rules: {rules}


## Process Steps:
1. Examine the input data for the 4 keys. 
2. Carefully consider EACH of the provided rules. 
3. Determine a classification based on comparing the input data against all rules - including the example for 'Other', as outlined above. If you can proceed with determining a classification, specifically for Categories, use the Exacting language noted in the rules. Note, for subgroup, apply consistency in selecting the values.
4. IMPORTANT: If you determine multiple rules apply or conflict, indicate the secondary, tertiary... through n, candidate changes in the 'candidates' k-v pair list

## Final Instructions:
Return the reclassified information according to the json schema provided.
Return a list of reclassified objects in the exact same order as provided in the input list. Each output record must be directly mapped to the input by index.

"""
def new_entity(text, cats, subgroups):
    response = oai_client.responses.parse(
        model="gpt-4.1",
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"For your additional reference in an effort to maintain consistency, here are the Entity Categories that have been reclassified so far: {cats}"},
            {"role": "user", "content": f"For your additional reference in an effort to maintain consistency, here are the Entity Categories that have been reclassified so far: {subgroups}"},
            {"role": "user", "content": f"Return your reclassification determinations for the input dictionaries: {text}"}   
        ],
        text_format = ReclassifyList,
        temperature=0
    )
    return response

#### Run the chunks

In [172]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [173]:
import pprint
for batch in chunk(ready_to_reclass_r2[:25], 5):
    batch_input = [f"#STARTRECORD# \n {json.dumps(dict(rec))} \n#ENDRECORD#" for rec in batch]
pprint.pprint(batch_input)


['#STARTRECORD# \n'
 ' {"original_id": 3053, "Name": "Tim LaMott Mechanical", "Entity Group ID": '
 '"Individual", "Entity Category": null, "Entity Subgroup ID": "Contractor"} \n'
 '#ENDRECORD#',
 '#STARTRECORD# \n'
 ' {"original_id": 3054, "Name": "David Stephenson Plumbing", "Entity Group '
 'ID": "Individual", "Entity Category": null, "Entity Subgroup ID": '
 '"Contractor"} \n'
 '#ENDRECORD#',
 '#STARTRECORD# \n'
 ' {"original_id": 3055, "Name": "Jon VanVleet Plumbing", "Entity Group ID": '
 '"Individual", "Entity Category": null, "Entity Subgroup ID": "Contractor"} \n'
 '#ENDRECORD#',
 '#STARTRECORD# \n'
 ' {"original_id": 3079, "Name": "U Save Excavation", "Entity Group ID": '
 '"Firm", "Entity Category": "Construction - Excavating", "Entity Subgroup '
 'ID": "Contractor"} \n'
 '#ENDRECORD#',
 '#STARTRECORD# \n'
 ' {"original_id": 3080, "Name": "M & B Electric LLC", "Entity Group ID": '
 '"Firm", "Entity Category": null, "Entity Subgroup ID": "Contractor"} \n'
 '#ENDRECORD#']


In [249]:
import time
import json
import logging
reclassified = []
responses = []
n = 0

# Setup logging
logging.basicConfig(
    filename="/Users/paigegiese/SYG/landproDATA_code/misc-work/db/entities_reclassed_v2.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

try:
    for batch in chunk(ready_to_reclass_r2[10:], 5):
        try:
            #batch
            batch_input = [f"#STARTRECORD# \n {json.dumps(dict(rec))} \n#ENDRECORD#" for rec in batch]
            # run batch
            reclassify_data = new_entity(text = batch_input, cats=categories, subgroups=subgroups)
            # append responses
            responses.append(reclassify_data)

            logging.info(f'Writing response to json file. # of responses: {len(responses)}')

            with open("responses_entities_reclassed_v2.jsonl", "a") as f:
                f.write(json.dumps(reclassify_data.model_dump()) + "\n")

            # view reclassifications
            data_new = reclassify_data.output_parsed.model_dump()['reclassified']
            # add to master list
            reclassified.extend(data_new)

            logging.info(f'Writing reclassified records to json file. # of reclassified records: {len(reclassified)}')

            with open("entities_reclassed_v2.jsonl", "a") as f:
                f.write(json.dumps(data_new) + "\n")

            n+=len(data_new)

            if n%50 == 0:
                print(f'On index {n}')
                logging.info(f'On index {n}')
        except Exception as e:
            print(f'Error:{e}')
            time.sleep(10) #trying the catch here in case we make too many api calls

except Exception as e:
    print(f'Error:{e}')
    time.sleep(10) #trying the catch here in case we make too many api calls

Error:1 validation error for ReclassifyList
  Invalid JSON: EOF while parsing a value at line 4 column 32774 [type=json_invalid, input_value='{\n  "reclassified": [\n...                       ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
On index 50
Error:1 validation error for ReclassifyList
  Invalid JSON: EOF while parsing a value at line 6 column 28066 [type=json_invalid, input_value='{\n  "reclassified": [\n...                       ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
On index 100
Error:1 validation error for ReclassifyList
  Invalid JSON: EOF while parsing a value at line 3 column 32774 [type=json_invalid, input_value='{"reclassified":[\n  {\n...                       ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
On index 150
On index 200
On index 250
On index 300
On index 350
On index 400
On index 450
On in

### Round 2 addins

In [251]:
reclassified

[{'original_id': '2711',
  'entity_group_id': 'Private',
  'entity_category': 'Contractor',
  'entity_subgroup_id': 'HVAC',
  'confidence': 98.0,
  'candidates': [{'entity_group_id': 'Private',
    'entity_category': 'Contractor',
    'entity_subgroup_id': 'General',
    'confidence': 70.0,
    'reasoning': "If 'Heating & Air' is not considered specific enough, could default to General Contractor, but HVAC is the standard specialty for this naming."}]},
 {'original_id': '2712',
  'entity_group_id': 'Private',
  'entity_category': 'Contractor',
  'entity_subgroup_id': 'Electrical',
  'confidence': 98.0,
  'candidates': [{'entity_group_id': 'Private',
    'entity_category': 'Contractor',
    'entity_subgroup_id': 'General',
    'confidence': 60.0,
    'reasoning': "If 'Electric' is not considered a specialty, could default to General, but standard practice is to use Electrical."}]},
 {'original_id': '3050',
  'entity_group_id': 'Private',
  'entity_category': 'Contractor',
  'entity_subg

In [259]:
reclassified_df = pd.DataFrame(reclassified)
reclassified_df = reclassified_df.loc[reclassified_df['original_id']!=', '].reset_index(drop=True)
reclassified_df['original_id'] = reclassified_df['original_id'].astype(int)

In [262]:
full_table['ID'] = full_table['ID'].astype(int)
full_table.merge(reclassified_df, how='inner', left_on='ID', right_on='original_id')

,original_id_x,entity_group_id_x,entity_category_x,entity_subgroup_id_x,confidence_x,candidates_x,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status,original_id_y,entity_group_id_y,entity_category_y,entity_subgroup_id_y,confidence_y,candidates_y
0,None,None,None,None,NaN,None,2711,Firm,Contractor,None,Carlon Heating & Air LLC,CARLON HEATING & AIR LLC,4.0,2711,Private,Contractor,HVAC,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."
1,None,None,None,None,NaN,None,2712,Firm,Contractor,None,Lee Electric,LEE ELECTRIC,4.0,2712,Private,Contractor,Electrical,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."
2,None,None,None,None,NaN,None,3050,Individual,Contractor,None,Mustafa Pajazetovic Electrical,None,4.0,3050,Private,Contractor,Electrical,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."
3,None,None,None,None,NaN,None,3051,Individual,Contractor,None,Brian Knott Electrical,Brian Knott Electrical,4.0,3051,Private,Contractor,Electrical,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."
4,None,None,None,None,NaN,None,3053,Individual,Contractor,None,Tim LaMott Mechanical,None,4.0,3053,Private,Contractor,Mechanical,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1811,None,None,None,None,NaN,None,16305,None,None,None,Female,None,2.0,16305,Private,Other,NULL,100.0,[]
1812,None,None,None,None,NaN,None,16307,None,None,None,Sevens Group,None,2.0,16307,Private,Other,NULL,100.0,[]
1813,None,None,None,None,NaN,None,16309,None,None,None,Self,None,2.0,16309,Private,Other,NULL,100.0,[]
1814,None,None,None,None,NaN,None,16312,None,None,None,AMP,None,2.0,16312,Private,Other,NULL,100.0,[]


In [263]:
full_table.to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/db/reclassified_061625.csv', index=False)

In [267]:
full_table['original_id']# = full_table['original_id'].astype(int)

0           1
1           2
2           4
3           5
4           6
         ... 
13390    None
13391    None
13392    None
13393    None
13394    None
Name: original_id, Length: 13395, dtype: object

In [268]:
reclassified_df.head(1)

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates
0,2711,Private,Contractor,HVAC,98.0,"[{'entity_group_id': 'Private', 'entity_catego..."


In [269]:
full_table.loc[full_table['ID'] == 2711]

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
2233,None,None,None,None,NaN,None,2711,Firm,Contractor,None,Carlon Heating & Air LLC,CARLON HEATING & AIR LLC,4.0


In [273]:
for i, j in reclassified_df.iterrows():
    full_table.loc[full_table['ID'] == j['original_id'],'original_id'] = j['original_id']
    full_table.loc[full_table['ID'] == j['original_id'],'entity_group_id'] = j['entity_group_id']
    full_table.loc[full_table['ID'] == j['original_id'],'entity_category'] = j['entity_category']
    full_table.loc[full_table['ID'] == j['original_id'],'entity_subgroup_id'] = j['entity_subgroup_id']
    full_table.loc[full_table['ID'] == j['original_id'],'confidence'] = j['confidence']
    #nullify the status so we can re-audit
    full_table.loc[full_table['ID'] == j['original_id'],'status'] = np.nan

In [275]:
full_table.loc[full_table['status'].isna()]['entity_category'].value_counts()

entity_category
Other                     732
Contractor                547
Real Estate               198
Builder                    83
Engineering                65
Agency                     60
Surveying                  38
Architect                  26
Inspection                 25
Appraisal                  13
Design                      9
Homeowners Association      5
Title                       4
Planning                    4
Energy                      2
Restoration                 2
Consulting                  2
Security                    1
Name: count, dtype: int64

In [ ]:
full_table

original_id            3197
entity_group_id        3197
entity_category        3197
entity_subgroup_id     4948
confidence             3591
candidates             5407
ID                        0
Entity Group ID         839
Entity Subgroup ID     2339
Entity Category       10096
Name                      1
IDSOS Name             1336
status                 1816
dtype: int64

In [ ]:
full_table['status'].replace('NULL',None).replace('', None)

In [283]:
full_table.loc[(full_table['status'].isna()) & ((full_table['entity_subgroup_id'] =='NULL')| (full_table['entity_subgroup_id'] =='')),'entity_subgroup_id'] = None

In [290]:
for i, j in full_table.iterrows():
    if j['entity_category'] not in categories:
        full_table.at[i,'entity_category'] = 'Other'

In [296]:
full_table.loc[(full_table['status'].isna()) & (full_table['entity_category']=='Other'),'status'] =6

In [301]:
pd.set_option('display.max_rows', 100)
full_table[full_table['status'].isna()]

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
2233,2711,Private,Contractor,HVAC,98.0,None,2711,Firm,Contractor,None,Carlon Heating & Air LLC,CARLON HEATING & AIR LLC,NaN
2234,2712,Private,Contractor,Electrical,98.0,None,2712,Firm,Contractor,None,Lee Electric,LEE ELECTRIC,NaN
2552,3050,Private,Contractor,Electrical,98.0,None,3050,Individual,Contractor,None,Mustafa Pajazetovic Electrical,None,NaN
2553,3051,Private,Contractor,Electrical,98.0,None,3051,Individual,Contractor,None,Brian Knott Electrical,Brian Knott Electrical,NaN
2554,3053,Private,Contractor,Mechanical,98.0,None,3053,Individual,Contractor,None,Tim LaMott Mechanical,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13382,16298,Private,Contractor,General,98.0,None,16298,Firm,None,None,SMITH CONSTRUCTION,None,NaN
13383,16299,Private,Contractor,General,98.0,None,16299,Firm,None,None,"SMITH CONSTRUCTION, INC.","SMITH CONSTRUCTION, INC.",NaN
13386,16302,Private,Real Estate,None,90.0,None,16302,None,None,None,Shoemaker Properties,None,NaN
13387,16304,Private,Engineering,Full Service,80.0,None,16304,None,None,None,LPD Forsgren Associates,None,NaN


In [ ]:
#"properties" --> mostly got tagged as Real Estate
# creating one audit for Agency category, Water subgroup (seems just a little sus)

full_table.loc[(full_table['entity_category']=='Agency') & (full_table['entity_subgroup_id']=='Water')\
    & (full_table['original_id'].isin(reclassified_df['original_id'].tolist())),'status'] = 6



In [308]:
full_table.loc[(full_table['status'].isna()),'status'] =7

In [310]:
full_table['status'].value_counts()

status
7.0    8203
0.0    1951
6.0    1637
1.0    1136
5.0     358
2.0      48
3.0      40
4.0      22
Name: count, dtype: int64

In [ ]:
# 0 = declined to process for quality issues (i.e. should be removed) .. includes Landpro records and inactives
# 1 = declined to process for 'developer' review = audit
# 2 = declined to process because of too little initial information (null,null) = audit
# 3 = declined to process because they were marked 'consultant' = audit 
# 4 = failed to process in first run -> needs re-run
# 5 = builder -> not builder = audit
# 6 = entity_category marked 'Other' = audit
# 7 = processed and leaving in... = no audit

In [322]:
full_table.loc[full_table['Name'].fillna('').str.lower().str.contains('onsultant') & full_table['Name'].fillna('').str.lower().str.contains('inactive'),'status'] =0

In [323]:
full_table.to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/db/reclassified_CLEAN_061625.csv', index=False)

In [324]:
full_table['status'].value_counts()


status
7.0    8203
0.0    1965
6.0    1637
1.0    1136
5.0     358
2.0      48
3.0      26
4.0      22
Name: count, dtype: int64

In [333]:
full_table[full_table['status']==7]


,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
0,1,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying,7.0
1,2,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.,7.0
2,4,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C.",7.0
3,5,Public,Agency,Road,99.0,"[{'entity_group_id': 'Public', 'entity_categor...",5,Agency,Public Agency,None,Ada County Highway District,None,7.0
4,6,Public,Agency,Land,95.0,"[{'entity_group_id': 'Public', 'entity_categor...",6,Agency,Public Agency,None,Ada County Surveyor's Office,None,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13383,16299,Private,Contractor,General,98.0,None,16299,Firm,None,None,"SMITH CONSTRUCTION, INC.","SMITH CONSTRUCTION, INC.",7.0
13384,16300,Private,Builder,Residential,NaN,None,16300,Firm,None,None,Sure Homes LLP,None,7.0
13386,16302,Private,Real Estate,None,90.0,None,16302,None,None,None,Shoemaker Properties,None,7.0
13387,16304,Private,Engineering,Full Service,80.0,None,16304,None,None,None,LPD Forsgren Associates,None,7.0


In [334]:
# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/landprodata")
query = "SELECT * FROM entities"
df = pd.read_sql(query, con=engine)
df.shape

(13397, 27)

In [335]:
full_table.shape

(13395, 13)

In [338]:
full_table['needs_audit'] = np.where(full_table['status'].isin([1,3,5,6]),1,0)

In [343]:
full_table_sorted = full_table[['Entity Group ID',
 'Entity Category',
 'Entity Subgroup ID',
 'Name',
 'IDSOS Name',
 'original_id',
 'entity_group_id',
 'entity_category',
 'entity_subgroup_id',
 'confidence',
 'candidates',
 'status',
 'needs_audit']].copy()

In [344]:
full_table_sorted.to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/db/reclassified_vDB_061625.csv', index=False)

In [347]:

from sqlalchemy import text
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/sandbox_paigeg")
with engine.connect() as conn:
    conn.execute(text( """CREATE TABLE tbl_entities_recategorized (
    id INT AUTO_INCREMENT PRIMARY KEY,

    `Entity Group ID` TEXT,
    `Entity Category` TEXT,
    `Entity Subgroup ID` TEXT,    
    `Name` TEXT,
    `IDSOS Name` TEXT,
    original_id INT,
    entity_group_id TEXT,
    entity_category TEXT,
    entity_subgroup_id TEXT,
    confidence INT,
    candidates TEXT,
    status INT,
    needs_audit INT
);"""))
    conn.commit()  # Needed for write operations in some engines

# cursor.execute(
#     "INSERT INTO my_table (id, data) VALUES (%s, %s)",
#     (1, json.dumps(mydict))
# )
# cnx.commit()


In [ ]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS tbl_entities_reclassified"))
    conn.close()

In [348]:
# Serialize dict columns if necessary
for col in ['candidates']:
    full_table_sorted[col] = full_table_sorted[col].apply(str)

In [349]:

# Insert into the table
full_table_sorted.to_sql(
    name='tbl_entities_recategorized',
    con=engine,
    if_exists='append',
    index=False
)

13395

## Impact

In [368]:
print(full_table['entity_group_id'].value_counts())
print(' ')
print(df['Entity Group ID'].value_counts())

entity_group_id
Private    9964
Public      234
Name: count, dtype: int64
 
Entity Group ID
Firm          12282
                335
Agency          181
Individual       79
Non-Entity       11
FIrmj             1
Builder           1
firm              1
Name: count, dtype: int64


In [361]:
full_table[full_table['status']==1]

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status,needs_audit
23,None,None,None,None,NaN,None,25,Firm,Developer,None,Boundaries Unlimited LLC,BOUNDARIES UNLIMITED LLC,1.0,1
25,None,None,None,None,NaN,None,27,Firm,Developer,Developer,Brighton Corporation,BRIGHTON CORPORATION,1.0,1
28,None,None,None,None,NaN,None,30,Firm,Developer,None,"C & MP Properties, LLC","C & MP PROPERTIES, LLC",1.0,1
64,None,None,None,None,NaN,None,70,Firm,Developer,None,Harris Family Ranch LLP,HARRIS FAMILY RANCH LLP,1.0,1
97,None,None,None,None,NaN,None,105,Firm,Developer,None,M & H Development LLC,M & H DEVELOPMENT LLC,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12725,None,None,None,None,NaN,None,15554,Firm,None,Developer,"Huter Land and Investments, LLC","Huter Land and Investments, LLC",1.0,1
12730,None,None,None,None,NaN,None,15564,Firm,Developer,None,"C, R, M & M LLC","C, R, M & M LLC",1.0,1
12823,None,None,None,None,NaN,None,15679,Firm,Developer,Developer,Seddie Eagle LLC,None,1.0,1
12826,None,None,None,None,NaN,None,15682,Firm,"Developer, Builder","Developer,Builder","Epic Development Idaho, LLC","Epic Development Idaho, LLC",1.0,1


,entity_category,count
0,Developer,1136


In [384]:
cats_bar = pd.concat([pd.DataFrame(full_table['entity_category'].value_counts()).reset_index(),pd.DataFrame([['Developer',1136]], columns=['entity_category','count'])])
cats_bar.sort_values(by='count')#.set_index('entity_category').plot.barh()

,entity_category,count
11,Title,20
10,Homeowners Association,29
9,Inspection,75
8,Appraisal,147
7,Architect,160
6,Agency,256
5,Surveying,329
4,Engineering,471
3,Builder,892
2,Real Estate,950


In [371]:
print(full_table['entity_category'].value_counts())
print('')
print(df['Entity Category'].nunique())

entity_category
Contractor                5339
Other                     1530
Real Estate                950
Builder                    892
Engineering                471
Surveying                  329
Agency                     256
Architect                  160
Appraisal                  147
Inspection                  75
Homeowners Association      29
Title                       20
Name: count, dtype: int64

105


In [386]:
print(full_table['entity_subgroup_id'].nunique())
print('')
print(df['Entity Subgroup ID'].nunique
())

218

152


In [392]:
pd.set_option('display.max_rows', 300)
full_table[['entity_group_id','entity_category','entity_subgroup_id']].value_counts().reset_index().to_csv('count.csv', index=False)